D'abord : se logguer à un compte gmail \
login : eleveenmi@gmail.com  (i=1,2,3,..) \
mwd : pwdeleveEnmi (i=1,2,3,..)

# I) Installations et imports

In [1]:
# Google sheet
!pip install --upgrade gspread gspread_dataframe

In [10]:
# Imports
from google.colab import auth
from google.auth import default
import gspread
from google.colab import drive
from google.colab import output
import os
import re
import shutil
import tarfile
from pathlib import Path
import random
from IPython.display import display, clear_output, Javascript
import ipywidgets as widgets
from base64 import b64encode
from datetime import datetime
import pandas as pd
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import re
# Spécifique au TP :
! git clone https://github.com/nanopiero/intro-ia-secondes-2025.git
! cp intro-ia-secondes-2025/utils/annotation.py .
! cp intro-ia-secondes-2025/utils/configuration.py .
# from google.colab import files
# uploaded = files.upload()
from importlib import reload
import configuration as cf
import annotation as an
# reload(cf)
# reload(an)

fatal: destination path 'intro-ia-secondes-2025' already exists and is not an empty directory.


<module 'annotation' from '/content/annotation.py'>

# II) Authentification

In [7]:
cf.group2sheets

{0: 'https://docs.google.com/spreadsheets/d/1sjvrkLMEY0YXAyXU9Ad8i49J29jCmKkJb85T2pZ2CS8',
 1: 'https://docs.google.com/spreadsheets/d/1sjvrkLMEY0YXAyXU9Ad8i49J29jCmKkJb85T2pZ2CS8',
 2: 'https://docs.google.com/spreadsheets/d/1Npi7H-Y-P60QBODOL8Q5ArQHr5dIn9mOuYv67LLHcNE',
 3: 'https://docs.google.com/spreadsheets/d/1I_ZwJrPt1AO_tQJSX2JBreFFhAlsF7kITH9p_1gDhks',
 4: 'https://docs.google.com/spreadsheets/d/1e1LnjbLKoZ_JxYfGLu1nFIQWyioNt4AV_Y9C1CUqiWI',
 5: 'https://docs.google.com/spreadsheets/d/1unaaeXdGVdC_1JMMVSBUgxc6vdrJzE16lTiP6OBEl04',
 6: 'https://docs.google.com/spreadsheets/d/1odXOBEiNbFQ48NALzTEmCs7LDlBHWARdsivL8rO47fw',
 7: 'https://docs.google.com/spreadsheets/d/1v7gJmdK1AqMelBRaksIcrIN_mDjobH8wxMxfLbiKlS8',
 8: 'https://docs.google.com/spreadsheets/d/1chlVgydwqpQYq44ynEWX_I0QVtWsUaYfwqVSptTD-dA',
 9: 'https://docs.google.com/spreadsheets/d/1x295asse_gh31-EzXQYFUSIkpbLe-hvYwWAxyr1c_ag',
 10: 'https://docs.google.com/spreadsheets/d/1QKZuVjBd1SOmAZvg1zA4vnzLnh8DXVzDouwA941WB2I'

In [3]:
# pour colab :
auth.authenticate_user()
# pour accéder aux fichiers google (google sheet)
creds, _ = default()  # or use service account
gc = gspread.authorize(creds)

# montage du drive :
drive.mount('/content/drive')

Mounted at /content/drive


# III) Dashboard

I've a dictionary of google sheets url on which I may use get_as_dataframe, set_with_dataframe. The keys are groups of annotators.
eg :

{0: 'https://docs.google.com/spreadsheets/d/1sjvrkLMEY0YXAyXU9Ad8i49J29jCmKkJb85T2pZ2CS8',
 1: 'https://docs.google.com/spreadsheets/d/1sjvrkLMEY0YXAyXU9Ad8i49J29jCmKkJb85T2pZ2CS8',
 2: 'https://docs.google.com/spreadsheets/d/1Npi7H-Y-P60QBODOL8Q5ArQHr5dIn9mOuYv67LLHcNE',

 In these google sheets, there are data like these:

  
Image,Classe,Hauteur,Nebul
O_202503080900.jpg,30,70,4
S_202503010900.jpg,4,,
S_202503011200.jpg,3,,
S_202503011500.jpg,3,,
S_202503011800.jpg,20,,1
S_202503020900.jpg,81,,8
S_202503021200.jpg,75,,
S_202503021500.jpg,75,,5

Please focus on the column Classe (don't take Hauteur and Nebul into account)

I want a function that takes as arguments:
-  a list of groups of ids
-  the dict of sheets url
and launch a widget with a large html table that will serve as a dashboard.
On that dashboard:
- list groups in the first row (groupe id1 id2 id3 etc) in the natural order (write groupe like in french)
- count the number of entries in the second row and fill in red the case where it is the max of counts, in blue where it is min of counts. head of the line will be "Labels"
- count the number of observed rare Class (Classe in [71, 72, 73, 41, 42, 43, 23, 44, 100]) and list them in a line filled in pale pink. Head of the line "Classes rares".
- add a last column "total", and give the total counts for each lines.


# III) Configuration

In [ ]:
group=13
direction, starthour = group2direction_and_starthour[group]
image_dir = f'{group}_{direction}_{starthour}'
tar_and_zipped_image_dir = image_dir + '.tar.gz'
starting_image_idx = 3

# IV) Récupération des images :

In [ ]:
# 1. Define the source path (change SHARED_PATH accordingly)
shared_path = '/content/drive/MyDrive/' + tar_and_zipped_image_dir
dest_path = Path.home() / tar_and_zipped_image_dir

# Copy the file to your home directory
shutil.copy(shared_path, dest_path)

# 2. Extract and delete the archive
with tarfile.open(dest_path, 'r:gz') as tar:
    tar.extractall(path='.')  # extract to current working directory

# 3. Remove the tar.gz file
dest_path.unlink()

# V) Récupération d'une partie des labels :

In [ ]:
# Open by ID or URL
sheet_url = group2sheets[group]
sh = gc.open_by_url(sheet_url)
worksheet = sh.sheet1